In [1]:
import esper.face_landmarks_wrapper as flw
from query.models import Face

In [12]:
faces = Face.objects.filter(
    frame__video_id__in=[123, 186, 188, 377], frame__regularly_sampled=True).order_by('id').all()

In [13]:
faces_that_have_landmarks = flw.exists(faces)

In [14]:
faces.count()

43408

In [15]:
faces_without_landmarks = []
for f, has_landmark in zip(faces, faces_that_have_landmarks):
    if not has_landmark:
        faces_without_landmarks.append(f.id)

In [16]:
len(faces_without_landmarks)

0

In [9]:
faces_without_landmarks[0]

8912968

In [10]:
faces_without_landmarks[-1]

8914518

In [9]:
faces_without_landmarks

[8912955,
 8912956,
 8912957,
 8912958,
 8912959,
 8912960,
 8912961,
 8912962,
 8912963,
 8912964,
 8912965,
 8912966,
 8912967]

In [12]:
vids_missing_landmarks = set()

In [13]:
for f in Face.objects.filter(id__in=faces_without_landmarks).annotate(video_id=F('frame__video_id')):
    vids_missing_landmarks.add(f.video_id)

In [14]:
len(vids_missing_landmarks)

23

In [15]:
vids_missing_landmarks

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23}

In [2]:
faces_in_1 = Face.objects.filter(frame__video_id__lte=23).order_by('id')

In [3]:
faces_in_1.count()

259369

In [4]:
faces_in_1_have_landmarks = flw.exists(faces_in_1)

In [5]:
faces_in_1_without_landmarks = [f.id for f, has_landmark in zip(faces_in_1, faces_in_1_have_landmarks) if not has_landmark]

In [6]:
len(faces_in_1_without_landmarks)

0

In [7]:
import numpy as np

In [10]:
np.frombuffer(FaceLandmarks.objects.get(id=1).landmarks, dtype=np.float64)

array([0.60104167, 0.21446078, 0.60260417, 0.23651961, 0.60572917,
       0.25490196, 0.60729167, 0.2745098 , 0.61041667, 0.29289216,
       0.61354167, 0.30759804, 0.61875   , 0.31617647, 0.6234375 ,
       0.32352941, 0.634375  , 0.32720588, 0.64583333, 0.31985294,
       0.65520833, 0.31617647, 0.66302083, 0.30759804, 0.66979167,
       0.28921569, 0.67291667, 0.27083333, 0.67604167, 0.25122549,
       0.67916667, 0.22916667, 0.68229167, 0.20710784, 0.60572917,
       0.19485294, 0.60885417, 0.1875    , 0.61354167, 0.18382353,
       0.61875   , 0.18382353, 0.6234375 , 0.1875    , 0.640625  ,
       0.1875    , 0.64583333, 0.18382353, 0.65208333, 0.18014706,
       0.65833333, 0.18382353, 0.66510417, 0.19117647, 0.6328125 ,
       0.21078431, 0.63125   , 0.2254902 , 0.6296875 , 0.24019608,
       0.63125   , 0.25122549, 0.628125  , 0.26348039, 0.6296875 ,
       0.26348039, 0.6328125 , 0.26715686, 0.6359375 , 0.26348039,
       0.6390625 , 0.26348039, 0.61354167, 0.21446078, 0.61666

In [19]:
len(np.frombuffer(FaceLandmarks.objects.get(id=1).landmarks, dtype=np.float64).tobytes())

1088

In [1]:
import os
import numpy as np
from rs_embed import EmbeddingData

LANDMARKS_DIR = '/app/data/landmarks'
LANDMARKS_PATH = os.path.join(LANDMARKS_DIR, 'landmarks_binary_sorted.bin')
ID_PATH = os.path.join(LANDMARKS_DIR, 'landmarks_ids_sorted.bin')
LANDMARKS_DIM = 272

In [4]:
def _load():
    id_file_size = os.path.getsize(ID_PATH)
    assert id_file_size % 8 == 0, \
        'Id file size is not a multiple of sizeof(u64)'
    n = int(id_file_size / 8)
    emb_file_size = os.path.getsize(LANDMARKS_PATH)
    assert emb_file_size % 4 == 0, \
        'Embedding file size is a multiple of sizeof(f32)'
    d = int((emb_file_size / 4) / (id_file_size / 8))
    assert emb_file_size % d == 0, \
        'Embedding file size is a multiple of d={}'.format(d)
    emb_data = EmbeddingData(ID_PATH, LANDMARKS_PATH, LANDMARKS_DIM)
    assert emb_data.count() == n, \
        'Count does not match expected: {} != {}'.format(n, emb_data.count())
    return emb_data

In [5]:
_LANDMARKS_DATA = _load()

In [8]:
_, binary = _LANDMARKS_DATA.get([3])[0]

In [22]:
type(binary[0])

float

In [24]:
np.frombuffer(np.array(binary, dtype=np.float32).tobytes(), dtype=np.float64)

array([0.2265625 , 0.72058824, 0.22291667, 0.75      , 0.22291667,
       0.77573529, 0.21927083, 0.80637255, 0.2265625 , 0.83578431,
       0.23020833, 0.85294118, 0.23020833, 0.86519608, 0.23177083,
       0.87009804, 0.2390625 , 0.87867647, 0.24479167, 0.87867647,
       0.25520833, 0.87377451, 0.26614583, 0.86151961, 0.2734375 ,
       0.84436275, 0.27916667, 0.81862745, 0.27916667, 0.80147059,
       0.28072917, 0.77573529, 0.2828125 , 0.74632353, 0.23385417,
       0.67769608, 0.228125  , 0.66421569, 0.22447917, 0.66053922,
       0.22447917, 0.66421569, 0.22447917, 0.66911765, 0.23020833,
       0.68137255, 0.23177083, 0.67769608, 0.23541667, 0.68137255,
       0.24114583, 0.68995098, 0.24635417, 0.70343137, 0.22447917,
       0.71568627, 0.21927083, 0.7377451 , 0.21197917, 0.75      ,
       0.20989583, 0.76715686, 0.22083333, 0.78063725, 0.21927083,
       0.78063725, 0.21927083, 0.78431373, 0.22291667, 0.78431373,
       0.2265625 , 0.78431373, 0.23020833, 0.70343137, 0.22656

In [25]:
np.frombuffer(FaceLandmarks.objects.get(face_id=3).landmarks, dtype=np.float64)

array([0.22604167, 0.78186275, 0.23072917, 0.78921569, 0.22916667,
       0.78553922, 0.21927083, 0.80514706, 0.22604167, 0.83210784,
       0.22916667, 0.85416667, 0.22916667, 0.8627451 , 0.23072917,
       0.86642157, 0.2359375 , 0.87009804, 0.2390625 , 0.87009804,
       0.24583333, 0.85416667, 0.26510417, 0.8125    , 0.21927083,
       0.8125    , 0.21770833, 0.8002451 , 0.22916667, 0.83210784,
       0.22083333, 0.79656863, 0.2765625 , 0.80882353, 0.28958333,
       0.74264706, 0.2       , 0.67401961, 0.19635417, 0.67769608,
       0.19791667, 0.67769608, 0.2359375 , 0.83210784, 0.22291667,
       0.78553922, 0.22604167, 0.78186275, 0.22447917, 0.78186275,
       0.275     , 0.78553922, 0.28177083, 0.76960784, 0.22291667,
       0.72794118, 0.21614583, 0.74264706, 0.21145833, 0.75857843,
       0.209375  , 0.76593137, 0.22083333, 0.78553922, 0.22083333,
       0.78921569, 0.22083333, 0.79289216, 0.22291667, 0.79289216,
       0.22604167, 0.79289216, 0.28489583, 0.77328431, 0.28489

In [11]:
from esper.prelude import *
from query.models import Face
from esper.stdlib import face_landmarks_to_dict, simple_result
import esper.face_landmarks_wrapper as flw
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.parsers import in_array
from rekall.merge_ops import payload_plus
from esper.rekall import intrvllists_to_result_with_objects

STRIDE = 1000
LIMIT = 100

# Face landmarks are keyed by Face ID's.
faces_qs = Face.objects.annotate(
    min_frame=F('frame__number'),
    max_frame=F('frame__number'),
    video_id=F('frame__video_id')).filter(video_id=24)

# Use coalesce to get a list of frames we want
# We store Video ID and frame number in the payload
frames = VideoIntervalCollection.from_django_qs(
    faces_qs[:LIMIT*STRIDE:STRIDE],
    with_payload=lambda face_obj: (face_obj.video_id, face_obj.min_frame)
).coalesce()

# face_landmarks_wrapper.get takes in a Face queryset or list of Face
#   objects and returns a list of LandmarksWrapper objects.
landmarks = frames.map(lambda interval: (
    interval.start, interval.end,
    flw.get(faces_qs.filter(
        video_id=interval.payload[0],
        min_frame=interval.payload[1]).all())
))

# We use face_landmarks_to_dict to draw LandmarksWrapper objects.
esper_widget(intrvllists_to_result_with_objects(
    landmarks,
    lambda landmarks_wrappers, video_id: [face_landmarks_to_dict(wrapper) for wrapper in landmarks_wrappers]
), crop_bboxes=False)

VGridWidget(jsglobals={'bucket': 'esper', 'queries': [['All faces', 'def all_faces():\n    from query.models i…